# pyisotopomer

pyisotopomer is a Python toolbox for performing data corrections for N2O isotopomer data. Its core is a package of scripts to correct for scrambling in the ion source during isotope ratio mass spectrometry.

Here, we are using a platform called Google Colab, which is free and allows you to run Python without installing it locally on your computer. Note that to run a cell you need to press SHIFT+ENTER. To edit a cell, you just need to double-click on it. The first few steps of this notebook are there specifically to set up the Google Colab environment, so if you're running this notebook locally on your computer, skip to the section entitled "Scrambling Calibration".

This Google Colab notebook and the repository containing it has “Read Only” access. In order to be able to modify the notebook and do your own data processing, you need to:
1. Access the [pyisotopomer_examples folder](https://drive.google.com/drive/folders/1XNmi0alJtopnELIcx8kcD2KBwru8kSLw?usp=sharing)
2. Right click on this notebook ("run_pyisotopomer.ipynb") and choose "add to my drive".
3. Navigate to the location of this folder in your Drive, right-click on it, and choose “Make a Copy”. It will create a new notebook entitled "Copy of pyisotopomer_examples". Rename/move it as you wish.
4. If you have never used Google Colab before, you need to connect Google Colab to your Google Drive. To do that, double-click on your run_pyisotopomer.ipynb, at the top of the page click on “Connect More Apps” and choose “Colab”. Sometimes, you need to load the webpage a few times before you see Google Colab in the app choices.

First, run the cell below to install the necessary packages in this Google Colab environment (this may take a couple of minutes).

In [1]:
%%capture
!pip install pandas numpy scipy
!pip install pyisotopomer

Then, we need to give this notebook access to Google Drive to access the data files. We'll run the cell below and follow the instructions to log in:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Now, _modify_ the text in quotes below, after cd, so that it reflects the name of your folder containing this notebook and your data corrections template:

In [ ]:
cd "drive/MyDrive/N2O Research/N2O_isotopocule_data_corrections/pyisotopomer/pyisotopomer_examples/"

[Errno 2] No such file or directory: 'drive/MyDrive/N2O Research/N2O_isotopocule_data_corrections/pyisotopomer/pyisotopomer_examples/'
/content


# Scrambling calibration

Here, two coefficients, γ and κ, are used to describe scrambling in the ion source. This is described in further detail in [Frame and Casciotti, 2010](https://www.biogeosciences.net/7/2695/2010/). Below is a description of how to calculate these coefficients in pyisotopomer.

Run two (or more) reference gases with known <sup>15</sup>R-α and <sup>15</sup>R-β, prepared in the same format as samples. For the Casciotti lab, this is some amount of N<sub>2</sub>O reference gas injected into a bottle of seawater or DI water that has been purged with He or N2 gas.

Download the data correction [template](https://docs.google.com/spreadsheets/d/1YMyaIXTMbX56r5_1C3ovAb_GBVsQJXfr/edit?usp=sharing&ouid=104573000701514802850&rtpof=true&sd=true). Pre-process the data according to the pyisotopomer [README](https://github.com/ckelly314/pyisotopomer). Save it into your current working directory.

Go to the "scale_normalization" tab of the excel template. Make sure it contains all of the reference materials you will use for the scrambling calculation. In column A "ref_tag", enter the names of your reference materials — these can be whatever you want, as long as they contain only alphanumeric characters and are short (15 characters or fewer).

Go to the "size_correction" tab of the excel template. To mark which rows of the correction template represent reference materials, in column B "ref_tag", add the names of the reference materials, as they appear in the "scale_normalization" tab. For example, here, atmosphere-equilibrated seawater is named "ATM" and is marked as such in the "ref_tag" column.

DO NOT MODIFY COLUMN HEADERS IN THE CORRECTION TEMPLATE. Save the correction template into your current working directory.

### Choice of Method

Pyisotopomer contains two methods for the calculation of γ and κ: a direct calculation of γ and κ based on the algebraic re-arrangement of the equation for <sup>31</sup>R, and a least-squares solver method. Given the right reference materials, it is not necessary to use a numerical algorithm to solve for γ and κ; algebraic manipulations can provide exact and unique solutions for each coefficient. This algebraic solution is the default method of calculation of γ and κ in pyisotopomer. It should be noted that this algebraic solution produces consistent results only when the site preferences of the two reference materials used in the calculation are sufficiently distinct — otherwise, it will return values of γ and κ that vary widely and may not fall in the range of plausible values (i.e., they are either negative or greater than one).

In the event that the user has yet to obtain reference materials that are different enough in their site preferences to produce consistent results with the algebraic method, the least squares approach can be used as a temporary alternative, with the following caveats and modifications:
 - The least squares solver finds a local minimum close to the initial guess for γ and κ. As such, if the solver is fed an initial guess other than the absolute minimum calculated from the algebraic solution, it will find the “wrong” scrambling coefficients.
 - Using these “wrong” scrambling coefficients is OK if your unknowns are close in their delta values to those of the reference materials, but will have a deleterious effect as the unknowns diverge in their isotopomer values from the reference materials.
 - If the least squares solver is fed the correct γ and κ as an initial guess, it will converge on that solution — although this is still not as robust as simply running reference materials of sufficiently distinct site preferences and calculating γ and κ algebraically.

The import convention for pyisotopomer is (hold down shift-enter to run the cell):

In [ ]:
from pyisotopomer import Scrambling, Isotopomers

The algebraic method is the default and does not require any modifications to the call to the Scrambling function. To change to the least squares method, specify it with the "method" keyword argument. If you have an _a priori_ initial guess for γ and κ, enter it using the "initialguess" keyword argument.

To calculate scrambling with the algebraic method, modify the "inputfile" keyword to reflect the name of your excel data corrections spreadsheet, then run the cell below by pressing shift-enter:

In [ ]:
Scrambling(inputfile="00_Python_template_v2.xlsx")

output saved as 220629_scrambling_output.xlsx
Difference in reference materials SPs may be too small for consistent results with the algebraic method.
Try setting method='least_squares'
SP1 - SP2 = 18.68 per mille
Difference in reference materials SPs may be too small for consistent results with the algebraic method.
Try setting method='least_squares'
SP1 - SP2 = 19.26 per mille
Difference in reference materials SPs may be too small for consistent results with the algebraic method.
Try setting method='least_squares'
SP1 - SP2 = 0.58 per mille


<Gamma: 1.2, Kappa: 1.127>

To calculate scrambling with the least squares method, modify the "inputfile" keyword to reflect the name of your excel data corrections spreadsheet, set the method to "least_squares", and enter an initial guess for gamma and kappa (format [gamma, kappa]). Run the cell below by pressing shift-enter:

In [ ]:
Scrambling(inputfile="00_Python_template_v2.xlsx", method="least_squares",
          initialguess=[0.17, 0.08])

output saved as 220629_scrambling_output.xlsx


<Gamma: 0.1723, Kappa: 0.07975>

The Scrambling function will create an output file entitled ```{date}_scrambling_output.xlsx``` with scrambling output, similar to this [example spreadsheet](https://docs.google.com/spreadsheets/d/1Z_jMqslWt4LfdaFTM_Ngt3a2VtxXn-mm/edit?usp=sharing&ouid=104573000701514802850&rtpof=true&sd=true). The Scrambling function will also output two .csv files containing intermediate data products: [normalized_ratios.csv](https://drive.google.com/file/d/1baG9H-MQuVRv9crKAKPQlj2wrp3l4qvj/view?usp=sharing) contains the <sup>15</sup>R<sup>bulk</sup>, <sup>17</sup>R, and <sup>18</sup>R that pyisotopomer calculated from the normalized <sup>45</sup>R and <sup>46</sup>R of each reference material, and [normalized_deltas.csv](https://drive.google.com/file/d/1bx-Mop1dzjX5rhooWN79dgdfTjhWOvUi/view?usp=sharing) contains the equivalent delta values. You can copy these delta values into Columns AT-AV. If the scale normalization was effective, the δ<sup>15</sup>N<sup>bulk</sup> and δ<sup>18</sup>O of each reference material should be close to their calibrated values; if not, you may need to check for problem reference materials.

# Calculating isotopomers

Size-correct your data (including all samples and standards), as as described in the pyisotopomer [README](https://github.com/ckelly314/pyisotopomer). Enter the appropriate γ and κ in Columns AP and AQ of the excel template. These should be a running average of γ and κ, calculated using a window of \~10 reference material pairings (see below).

DO NOT MODIFY COLUMN HEADERS IN THE CORRECTION TEMPLATE. Save the correction template into your current working directory.

### How to think about scrambling when calculating isotopomers

You will need to enter the appropriate scrambling coefficients in the excel template. These scrambling coefficients should represent a running average of γ and κ calculated from at least 10 pairings of reference materials (e.g. a week's worth, if unknowns are bookended by reference materials) run alongside unknowns. This is because a small standard deviation in the scrambling coefficients can lead to a large error in site preference, so it is advisable to run sufficient reference materials to bring down the standard deviation of γ and κ. To calculate these running averages, it can be helpful to keep a spreadsheet with a running log of scrambling coefficients.

To calculate isotopomers, modify the "inputfile" keyword to reflect the name of your excel data corrections spreadsheet, then run the cell below by pressing shift-enter:

In [ ]:
Isotopomers(inputfile = "00_Python_template_v2.xlsx")  

output saved as 220629_isotopeoutput.csv


< First row:
d15Na: 15.11
d15Nb: -2.71
d15Nbulk: 6.198
SP: 17.82
d18O: 47.05>
                

The Isotopomers function will create an output file entitled ```{date}_isotopeoutput.csv``` with isotopocule delta values, similar to this [example spreadsheet](https://drive.google.com/file/d/1ZWws_32rjzutNkmD4HYebJBWjjIPRwt1/view?usp=sharing). Copy and paste output data back into working (size correction) spreadsheet in olive-highlighted cells (columns AX-BC).

# Go further

To learn about additional keyword arguments and different ways of accessing the outputs, run the following cells:

In [ ]:
?Scrambling

In [ ]:
?Isotopomers